Set the environment variable for the Groq API key to authenticate requests.

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "[YOUR_GROQ_API_KEY]"

Install the necessary Python packages: `langchain-groq`, `langchain-core`, and `requests`.

In [2]:
!pip install -q langchain-groq langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


Import the required modules from LangChain and the `requests` library.

In [3]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

Create a simple tool using the `@tool` decorator that multiplies two numbers.

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

Invoke the `multiply` tool with sample arguments to test its functionality.

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


Display the name of the `multiply` tool.

In [6]:
multiply.name

'multiply'

Display the description of the `multiply` tool.

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

Display the argument schema for the `multiply` tool.

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
# tool binding

Create a ChatGroq language model instance with a specified model and temperature.

In [10]:
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)

Invoke the LLM with a simple greeting to test its response.

In [11]:
llm.invoke('hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi". We need to respond. The instruction: "You are ChatGPT, a large language model trained by OpenAI." There\'s no special instruction. Just respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 72, 'total_tokens': 129, 'completion_time': 0.058765177, 'prompt_time': 0.003969963, 'queue_time': 0.212691016, 'total_time': 0.06273514}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--75961398-481d-476c-b5c7-a8711bd6ce1e-0', usage_metadata={'input_tokens': 72, 'output_tokens': 57, 'total_tokens': 129})

Bind the `multiply` tool to the LLM so it can use the tool during conversations.

In [12]:
llm_with_tools = llm.bind_tools([multiply])

Invoke the LLM with tools enabled using a simple greeting.

In [13]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hello! I'm just a bunch of code, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'reasoning_content': 'User says "Hi how are you". We respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 133, 'total_tokens': 186, 'completion_time': 0.0550846, 'prompt_time': 0.006773302, 'queue_time': 0.211471088, 'total_time': 0.061857902}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f8622f6-fe9b-468a-82d8-f4933f531261-0', usage_metadata={'input_tokens': 133, 'output_tokens': 53, 'total_tokens': 186})

Create a `HumanMessage` asking the LLM to multiply two numbers.

In [14]:
query = HumanMessage('can you multiply 3 with 1000')

Initialize a list of messages to be used in the conversation.

In [15]:
messages = [query]

Display the current list of messages.

In [16]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

Invoke the LLM with the prepared messages and tools.

In [17]:
result = llm_with_tools.invoke(messages)

Append the AI's response to the messages list.

In [18]:
messages.append(result)

Display the updated list of messages after appending the AI response.

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function.', 'tool_calls': [{'id': 'fc_925880de-aedf-49c6-b978-6d289ad6746e', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 138, 'total_tokens': 174, 'completion_time': 0.036223392, 'prompt_time': 0.006609553, 'queue_time': 0.212023606, 'total_time': 0.042832945}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_c5a89987dc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d089ad3d-4bc1-4a0e-a55a-e1ba3d463fbd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_925880de-aedf-49c6-b978-6d289ad6746e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 36, 'total_tokens': 174})]

Invoke the `multiply` tool using the arguments extracted from the AI's tool call.

In [20]:
tool_result = multiply.invoke(result.tool_calls[0])

Display the result returned by the `multiply` tool.

In [21]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='fc_925880de-aedf-49c6-b978-6d289ad6746e')

Append the tool result to the messages list.

In [22]:
messages.append(tool_result)

Display the final list of messages after all steps.

In [23]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function.', 'tool_calls': [{'id': 'fc_925880de-aedf-49c6-b978-6d289ad6746e', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 138, 'total_tokens': 174, 'completion_time': 0.036223392, 'prompt_time': 0.006609553, 'queue_time': 0.212023606, 'total_time': 0.042832945}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_c5a89987dc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d089ad3d-4bc1-4a0e-a55a-e1ba3d463fbd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_925880de-aedf-49c6-b978-6d289ad6746e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 36, 'total_tokens': 174}),
 Tool

Invoke the LLM with the full conversation and display its final response.

In [24]:
llm_with_tools.invoke(messages).content

'Sure! 3 × 1000 = **3000**.'

Define two tools: one to fetch the currency conversion factor using an API, and another to convert a value using the conversion rate.

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


Display the argument schema for the `convert` tool.

In [26]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

Invoke the `get_conversion_factor` tool with sample currencies to test its output.

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1760486401,
 'time_last_update_utc': 'Wed, 15 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1760572801,
 'time_next_update_utc': 'Thu, 16 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.8152}

Invoke the `convert` tool with sample values to test its output.

In [28]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

Bind the currency conversion tools to a new LLM instance.

In [29]:
# tool binding
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)

Bind the `get_conversion_factor` and `convert` tools to the LLM.

In [30]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

Create a `HumanMessage` asking for the conversion factor and to convert a value between INR and USD.

In [31]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

Display the current list of messages for the currency conversion task.

In [32]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

Invoke the LLM with the currency conversion query and tools.

In [33]:
ai_message = llm_with_tools.invoke(messages)

Append the AI's response to the messages list.

In [34]:
messages.append(ai_message)

Display the tool calls suggested by the AI in its response.

In [35]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'fc_034e32db-cf22-4125-9cc1-a9bf4e5d71da',
  'type': 'tool_call'}]

Iterate through the tool calls, execute them, and update the messages list with the results.

In [36]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



Display the updated list of messages after executing the tool calls.

In [37]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use function get_conversion_factor to get factor between INR and USD. Then use convert to convert 10 INR to USD. The convert function expects base_currency_value: number. But we need to specify base_currency? The function signature: convert(_:{base_currency_value: number,}) => any. It doesn\'t specify target currency. Maybe the context is that the conversion factor is already known? Actually convert likely uses the previously fetched conversion factor. But we need to call get_conversion_factor first. Then call convert with base_currency_value: 10. But we need to pass target currency? The function signature doesn\'t include target_currency. Maybe the convert function uses the last fetched conversion factor. So we call get_conversion_factor with ba

Invoke the LLM with the full conversation and display its final response for the currency conversion task.

In [38]:
llm_with_tools.invoke(messages).content

'**Conversion factor (INR → USD):** 1\u202fINR = **0.01126\u202fUSD**\n\n**10\u202fINR in USD:**\n\n\\[\n10 \\text{\u202fINR} \\times 0.01126 \\frac{\\text{USD}}{\\text{INR}} = 0.1126 \\text{\u202fUSD}\n\\]\n\nSo, 10\u202fINR is approximately **$0.11 USD**.'

Initialize a LangChain agent using the structured chat zero-shot ReAct pattern and bind the currency tools.

In [39]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


Run the agent with a sample user query and display the response.

In [40]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"
}

> Finished chain.
